In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import os, sys
from typing import *
import torch
import random
import copy
import warnings

from sklearn.metrics import classification_report

In [2]:
# print(os.getcwd())

# for i in range (3):
#     os.chdir("..")
    
# main_data_dir = os.getcwd() + "/Data set"

In [3]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/SPH"
print(os.listdir(data_dir))

['metadata.csv', 'data_df.csv', 'data_df_no1.csv', 'records']


In [4]:
main_df = pd.read_csv(data_dir + "/data_df.csv")
main_df.shape

(20792, 4)

In [5]:
single_fns = main_df["File name"].values.tolist()
single_mat_paths = [data_dir + f"/records/{x}.h5" for x in single_fns]

In [6]:
# ratio = [0.9, 0.1]

# train_index = int(len(single_mat_paths)*ratio[0])

# train_mat_paths = single_mat_paths[:train_index]
# valid_mat_paths = single_mat_paths[train_index:]

# train_label = main_df
# valid_label = main_df

In [7]:
# single_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in single_fns]
# print(len(single_mat_paths))

In [8]:
ratio = [0.8, 0.1]

train_index = int(len(single_mat_paths)*ratio[0])
valid_index = int(len(single_mat_paths)*(ratio[0]+ratio[1]))

train_mat_paths = single_mat_paths[:train_index]
valid_mat_paths = single_mat_paths[valid_index:]

train_label = main_df
valid_label = main_df

In [9]:
import h5py

class HeartData(Dataset):
    def __init__(self, data_paths):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]
        a = h5py.File(data_path, 'r')
        data_h5 = a['ecg']
        data = np.array(data_h5)
        clip_data = data[:, 0:3000]

        filename = data_path.split("/")[-1].split(".")[0]
        label = main_df[main_df["File name"] == filename]["New Label"].values.item()

        torch_data = torch.from_numpy(clip_data)

        return torch_data.float(), label

    def __len__(self):
        return len(self.data_paths)

In [10]:
train_ds = HeartData(train_mat_paths)
valid_ds = HeartData(valid_mat_paths)

In [11]:
train_dl = DataLoader(train_ds, batch_size = 64, shuffle = True, pin_memory = True, num_workers = 48)
valid_dl = DataLoader(valid_ds, batch_size = 64, shuffle = True, pin_memory = True, num_workers = 48)

In [12]:
class Auto_Encoder(nn.Module):
    def __init__(self):
        super(Auto_Encoder, self).__init__()

        self.enc = nn.Sequential(    
            # 12 x 3000
            nn.Conv1d(12, 64, 4, 2, 1),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),
            # 64 x 1500
            nn.Conv1d(64, 64, 3, 2, 1),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),
            # 64 x 750
            nn.Conv1d(64, 128, 3, 2, 1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),
            # 128 x 375
            nn.Conv1d(128, 128, 3, 2, 1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),  
            # 128 x 188
        )
        
        self.cls = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(128, out_features=34, bias=True)
        )

        self.dec = nn.Sequential(
            # 128 x 188
            nn.ConvTranspose1d(128, 128, 3, 2, 1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),
            # 128 x 375
            nn.ConvTranspose1d(128, 64, 2, 2),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),
            # 64 x 750
            nn.ConvTranspose1d(64, 64, 3, 2, 1, 1),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            # nn.ReLU(), #Laeky relu 0.2
            # 64 x 1500
            nn.ConvTranspose1d(64, 12, 4, 2, 1),
            # 12 x 3000
            nn.Sigmoid()
        )


    def forward(self, x):
        enc = self.enc(x) # 128 x 7 x 7 
        cls = self.cls(enc.mean(dim=-1))
        dec = self.dec(enc)
        
        return dec, cls

In [13]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3): #Change gamma value here in order to acquire other results
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)

    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()

In [14]:
epoch = 150
lr = 0.0005
best_acc = 0
best_ep = 0
# class_la = [1,2,3,4,5,6,7,8,9]
# for i in range (len(class_la)):
#     class_la[i] = str(class_la[i])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 0)
# device = torch.device("cpu", index = 0)

model = Auto_Encoder().to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch)
loss_fn_cls = nn.CrossEntropyLoss()
loss_fn_sig = nn.MSELoss()

In [15]:
warnings.filterwarnings('ignore')
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    y_true_list = [] 
    pred_list = []
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(train_dl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        res_sig, pred_cls = model(train_sig)
        loss_cls = focalloss_fn(pred_cls, train_label)
        loss_sig = loss_fn_sig(res_sig, train_sig)
        loss_tot = loss_cls + loss_sig
        
        optimizer.zero_grad()
        loss_tot.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss_tot.item()
        correct += (pred_cls.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(train_dl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(valid_dl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            res_sig, pred_cls = model(valid_sig)
            loss_cls = loss_fn_cls(pred_cls, valid_label)
            loss_sig = loss_fn_sig(res_sig, valid_sig)
            loss_tot = loss_cls + loss_sig
            
            pred_pos = pred_cls.argmax(1)
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)

            val_total_loss += loss_tot.item()
            val_correct += (pred_cls.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(valid_dl.dataset)
        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")

        
y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()

reports = classification_report(y_true, pred, output_dict=True)

print(reports)
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

# 0.8091346153846154 at epoch 104

Epoch: 0


0it [00:00, ?it/s]

1it [00:12, 12.06s/it]

5it [00:12,  1.82s/it]

10it [00:12,  1.35it/s]

...

0it [00:00, ?it/s]

1it [00:00,  4.37it/s]

24it [00:00, 90.37it/s]

33it [00:00, 67.94it/s]

valid loss: 1.966730204410851 - valid acc: 78.75
{'0': {'precision': 0.8156739811912226, 'recall': 0.9468704512372634, 'f1-score': 0.8763893566857528, 'support': 1374.0}, '1': {'precision': 0.7441860465116279, 'recall': 0.6666666666666666, 'f1-score': 0.7032967032967034, 'support': 48.0}, '2': {'precision': 0.8243243243243243, 'recall': 0.46923076923076923, 'f1-score': 0.5980392156862746, 'support': 130.0}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 86.0}, '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 13.0}, '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3.0}, '6': {'precision': 0.8, 'recall': 0.4, 'f1-score': 0.5333333333333333, 'support': 10.0}, '7': {'precision': 0.8125, 'recall': 0.65, 'f1-score': 0.7222222222222223, 'support': 20.0}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2.0}, '9': {'precision': 0.2777777777777778, 'recall': 0.625, 'f1-score': 0.3846153846153846, 'support': 8.0}, '10': {'